In [1]:
import numpy as np
import scipy
from lightfm.datasets import fetch_stackexchange
from lightfm import LightFM, evaluation

C:\Users\maloq\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
# Fetch Q&A data from stackexchange
data = fetch_stackexchange('crossvalidated', test_set_fraction = 0.25, min_training_interactions = 10)
#The datasets contain users answering questions: an interaction is defined as a user answering a given question.
print(repr(data['train']))
print(repr(data['test']))

<661x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 40701 stored elements in COOrdinate format>
<661x72360 sparse matrix of type '<class 'numpy.float32'>'
	with 8108 stored elements in COOrdinate format>


### Dataset contains the followings:

    train (sp.coo_matrix of shape [n_users, n_items]) – Contains training set interactions.
    test (sp.coo_matrix of shape [n_users, n_items]) – Contains testing set interactions.
    item_features (sp.csr_matrix of shape [n_items, n_item_features]) – Contains item features.
    item_feature_labels (np.array of strings of shape [n_item_features,]) – Labels of item features.


In [3]:
# Create models
bpr_model = LightFM(loss='bpr')
warp_model = LightFM(loss='warp')
warp_kos_model = LightFM(loss='warp-kos', k = 3, n = 3)
# Train model 
# Epochs -> number of runs, num_threads -> parallel runs
bpr_model.fit(data['train'], epochs = 30, num_threads = 2) 
warp_model.fit(data['train'], epochs = 30, num_threads = 2) 
warp_kos_model.fit(data['train'], epochs = 30, num_threads = 2) 

In [4]:
# Measure the precision at k metric for a model: 
bpr_precision = evaluation.precision_at_k(bpr_model, data['test'], k=10).mean()
warp_precision = evaluation.precision_at_k(warp_model, data['test'], k=10).mean()
warp_kos_precision = evaluation.precision_at_k(warp_kos_model, data['test'], k=10).mean()

print('Precision of BPR model:', "{:.3%}".format(bpr_precision))
print('Precision of WARP model:', "{:.3%}".format(warp_precision))
print('Precision of k-OS WARP model:', "{:.3%}".format(warp_kos_precision))

Precision of BPR model: 0.088%
Precision of WARP model: 0.058%
Precision of k-OS WARP model: 0.088%


In [5]:
def multi_recommendation(model, data, user_ids):
    """ 
    Recommend threads for users based on their previous interactions.     
    """    
    
    # Number of users and threads in the training data
    n_users, n_items = data['train'].shape
    
    # Generate recommendation for each user we input
    for user_id in user_ids:
        
        # Threads they already posted
        known_positives = data['item_feature_labels'][data['train'].tocsr()[user_id].indices]
        # Posts our model predicts they could like
        scores = model.predict(user_id, np.arange(n_items))
        # Sort them for the most fitting to least
        # argsort creates an array of indices, lowest score is the best fit
        top_items = data['item_feature_labels'][np.argsort(-scores)] 
        
        # Print out the results
        print('User ', user_id)
        print('Known positives:')
        
        for x in known_positives[:3]:
            print('   ',x)
            
        print('Recommendation:')
        
        for x in top_items[:3]:
            print('   ',x)
            
        print('\n')

In [6]:
# Sample recommendations for user '25'

user_ids = [25]

print('BPR model recommendations:\n')
multi_recommendation(bpr_model, data, user_ids)
print('WARP model recommendations:\n')
multi_recommendation(warp_model, data, user_ids)
print('k-OS WARP model recommendations:\n')
multi_recommendation(warp_kos_model, data, user_ids)

BPR model recommendations:

User  25
Known positives:
    question_id:3
    question_id:20
    question_id:41
Recommendation:
    question_id:170
    question_id:69
    question_id:1067


WARP model recommendations:

User  25
Known positives:
    question_id:3
    question_id:20
    question_id:41
Recommendation:
    question_id:1121
    question_id:1293
    question_id:42


k-OS WARP model recommendations:

User  25
Known positives:
    question_id:3
    question_id:20
    question_id:41
Recommendation:
    question_id:1460
    question_id:259
    question_id:580


